In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler 
from datetime import timedelta
from statsmodels.tsa.stattools import acf 
import pmdarima as pm 
from sklearn.linear_model import LassoCV 
from sklearn.linear_model import Lasso
import statsmodels.api as sm

In [ ]:
production_data_loc = "data/production.csv"
weather_data_loc = "data/processed_weather.csv"

In [ ]:
production_df = pd.read_csv(production_data_loc)
production_df["date"] = pd.to_datetime(production_df["date"])
production_df = production_df.iloc[4:] 
production_df = production_df.drop_duplicates()
production_df.reset_index(drop=True, inplace=True) 
production_df["production"] = production_df["production"].apply(lambda x: 10 if x > 10 else x) 

production_df

In [ ]:
weather_df = pd.read_csv(weather_data_loc) 
weather_df["date"] = pd.to_datetime(weather_df["date"], format='%Y-%m-%d') 
weather_df = weather_df.sort_values(by=['date', 'hour'])
weather_df.reset_index(drop=True, inplace=True) 

weather_df

In [ ]:
columns_to_pivot = weather_df.columns[4:]

for col in columns_to_pivot:
    weather_df[f'{col}_identifier'] = col + "_" + weather_df['lat'].astype(str) + "_" + weather_df['lon'].astype(str)

pivoted_dfs = []
for col in columns_to_pivot:
    pivoted_df = pd.pivot(
        weather_df,
        index=['date', 'hour'],
        columns=f'{col}_identifier',
        values=col
    )
    pivoted_df.columns.name = None 
    pivoted_df.reset_index(inplace=True)  
    pivoted_dfs.append(pivoted_df)

result_df = pivoted_dfs[0]
for df in pivoted_dfs[1:]:
    result_df = result_df.merge(df, on=['date', 'hour'], how='outer') 
    
df = result_df.iloc[:, :252] 

df

In [ ]:
date = pd.to_datetime("2024-05-12")
df = df[df["date"] != date]
production_df = production_df[production_df["date"] != date]

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 10)) 

columns_to_scale = df.columns[2:] 

df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

In [ ]:
end_date = production_df["date"].iloc[-1]
end_hour = production_df["hour"].iloc[-1]

predict_df = df.copy() 

cut_off_index = df[(df["date"] == end_date) & (df["hour"] == end_hour)].index.min() 
df = df.loc[:cut_off_index]
df = pd.merge(df, production_df, on=["date", "hour"], how = "inner")
df.fillna(method='ffill', inplace=True)

df

In [ ]:
acf_values = acf(df["production"], nlags=24*10)
plt.figure(figsize=(20, 5))
plt.plot(range(len(acf_values)), acf_values, marker='o', linestyle='-')
for i in range(1, 11):
    plt.axvline(x=24*i, color='red', linestyle='--')
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.show()

In [ ]:
diff_24_production = [df["production"][i] - df["production"][i-24] for i in range(24, len(df["production"]))]

acf_values = acf(diff_24_production, nlags=24*10)
plt.figure(figsize=(20, 5))
plt.plot(range(len(acf_values)), acf_values, marker='o', linestyle='-')
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.show()

In [ ]:
arima_model = pm.auto_arima(
    diff_24_production,
    start_d=0,
    max_d=1,
    seasonal=False,  
    stepwise=True,   
    trace=True,      
    error_action='ignore',  
    suppress_warnings=True, 
    max_order=10,     
    information_criterion='bic',  
)

best_arima = arima_model.fit(diff_24_production)

In [ ]:
best_arima.summary()

In [ ]:
forecasted_values = list(df["production"][:24])

for diff in list(best_arima.fittedvalues()):
    new_value = forecasted_values[-24] + diff
    forecasted_values.append(new_value)

forecasted_values = [max(0, i) for i in forecasted_values]

plt.figure(figsize=(12, 8))
plt.plot(range(len(df["production"][:1000])),forecasted_values[:1000], label="Forecast", c = "red")
plt.plot(range(len(df["production"][:1000])),df["production"][:1000], label="Real", c="blue")
plt.title("Forecast vs Real Production(Last 1000)")
plt.legend(loc="best")
plt.show()

In [ ]:
features = df.copy()
features.drop(columns=["date", "hour", "production"], inplace=True)

lst = weather_df.columns[4:-10]

for feature in lst:
    feature_col = df.columns[df.columns.str.contains(feature)]
    features[f"max_{feature}"] = df[feature_col].max(axis=1)
    features[f"min_{feature}"] = df[feature_col].min(axis=1)
    features[f"mean_{feature}"] = df[feature_col].mean(axis=1)
    features[f"sum_{feature}"] = df[feature_col].sum(axis=1)
    features[f"median_{feature}"] = df[feature_col].median(axis=1)
    features[f"std_{feature}"] = df[feature_col].std(axis=1)
    features[f"var_{feature}"] = df[feature_col].var(axis=1)
    features[f"range_{feature}"] = df[feature_col].max(axis=1) - df[feature_col].min(axis=1)

features

In [ ]:
lasso_cv = LassoCV(cv=5)
lasso_cv.fit(features, df["production"])

optimal_lambda = lasso_cv.alpha_

optimal_lambda

In [ ]:
lasso = Lasso(optimal_lambda)

lasso.fit(features, df["production"])

In [ ]:
lasso_result = lasso.predict(features)
fitted = [max(0, i) for i in lasso_result]

plt.figure(figsize=(25, 12))
plt.plot(range(500, 2500), df["production"][500:2500], label="Real Values", color="blue")
plt.plot(range(500, 2500), fitted[500:2500], label="Fitted Values", color="red")
plt.legend(loc="best")
plt.show()

In [ ]:
df_combined = pd.DataFrame(data = [lasso_result, forecasted_values])
df_combined = df_combined.T
df_combined.columns = ['Lasso', 'Arima']
df_combined = sm.add_constant(df_combined)

df_combined

In [ ]:
lm = sm.OLS(df["production"], df_combined)
result = lm.fit()
print(result.summary())

In [ ]:
fitted = [max(0, i) for i in result.fittedvalues]
fitted = [min(10, i) for i in fitted]

plt.figure(figsize=(20, 8))
plt.plot(range(0, 500), df["production"][-500:], label="Real Values", color="blue")
plt.plot(range(0, 500), fitted[-500:], label="Fitted Values", color="red")
plt.legend(loc="best")
plt.show()

In [ ]:
result.resid.plot(kind="kde", title="Residuals", figsize=(10, 5))

In [ ]:
residual = result.resid

plt.figure(figsize = (12, 7))
plt.scatter(range(len(df)),residual, label = "Residual", color = "blue", s = 10)
plt.title("Residuals of the Model")

plt.show()

In [ ]:
plt.figure(figsize=(12.5, 7))  
plt.acorr(result.resid[:1000], maxlags=len(result.resid[:1000])-1, usevlines = False, marker='o')
plt.axhline(y=0.125, color='red', linestyle='--') 
plt.axhline(y=-0.125, color='red', linestyle='--') 
plt.title("Autocorrelation of Residuals")
plt.xlim(0, 1020)
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.show()

In [ ]:
cut_off_date = production_df["date"].iloc[-1]

next_day = cut_off_date + timedelta(days=2)

result_df = predict_df[predict_df["date"] == next_day]
features = result_df.copy()
features.drop(columns=["date", "hour"], inplace=True)
lst = weather_df.columns[4:-10]

for feature in lst:
    feature_col = result_df.columns[result_df.columns.str.contains(feature)]
    features[f"max_{feature}"] = result_df[feature_col].max(axis=1)
    features[f"min_{feature}"] = result_df[feature_col].min(axis=1)
    features[f"mean_{feature}"] = result_df[feature_col].mean(axis=1)
    features[f"sum_{feature}"] = result_df[feature_col].sum(axis=1)
    features[f"median_{feature}"] = result_df[feature_col].median(axis=1)
    features[f"std_{feature}"] = result_df[feature_col].std(axis=1)
    features[f"var_{feature}"] = result_df[feature_col].var(axis=1)
    features[f"range_{feature}"] = result_df[feature_col].max(axis=1) - result_df[feature_col].min(axis=1)

lasso_pred = lasso.predict(features)

In [ ]:
arima_res = best_arima.predict(n_periods = 24)
last_24_hours = production_df[-24:]

arima_pred = [last_24_hours["production"].iloc[i] + arima_res[i] for i in range(len(arima_res))]

In [ ]:
df_combined = pd.DataFrame(data = [lasso_pred, arima_pred])
df_combined = df_combined.T
df_combined.columns = ['Lasso', 'Arima']
df_combined = sm.add_constant(df_combined)

In [ ]:
prediction = result.predict(df_combined)
prediction = [max(0, i) for i in prediction]
prediction = [min(10, i) for i in prediction]
prediction = [round(i,2) for i in prediction]

In [ ]:
final_pred = [0] * 5 + prediction[5:16] + [0] * 8
formatted_list = ",".join(map(str, final_pred))

print(next_day)
print("\n")
print(formatted_list)